# Deep Learning

# Assignment 3

Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.
The goal of this assignment is to explore regularization techniques.

In [62]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in notmist.ipynb.

In [63]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [64]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset_reshaped, train_labels_reshaped = reformat(train_dataset, train_labels)
valid_dataset_reshaped, valid_labels_reshaped = reformat(valid_dataset, valid_labels)
test_dataset_reshaped, test_labels_reshaped = reformat(test_dataset, test_labels)
print('Training set', train_dataset_reshaped.shape, train_labels_reshaped.shape)
print('Validation set', valid_dataset_reshaped.shape, valid_labels_reshaped.shape)
print('Test set', test_dataset_reshaped.shape, test_labels_reshaped.shape)

Training set (20000, 784) (20000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


In [65]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## Problem 1
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.

### Logistic Model

In [8]:
from sklearn.linear_model import LogisticRegression
logisticRegression = LogisticRegression(penalty='l2',random_state=11)
logisticRegression.fit(train_dataset_reshaped, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=11, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
valid_preds = logisticRegression.predict(valid_dataset_reshaped)
np.mean(valid_preds == valid_labels)

0.81599999999999995

In [10]:
test_preds = logisticRegression.predict(test_dataset_reshaped)
np.mean(test_preds == test_labels)

0.88500000000000001

### Neural Network Model

In [54]:
batch_size = 128
hidden1_units = 1024#One hidden layer here
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset_reshaped)
  tf_test_dataset = tf.constant(test_dataset_reshaped)
  
  # Variables in hidden layer.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases_1 = tf.Variable(tf.zeros([hidden1_units]))
  
  # Training computation.
  logits1= tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden1 = tf.nn.relu(logits1)
  
  #Variables in output layer
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  logits2 = tf.matmul(hidden1, weights_2) + biases_2

 #Regularization
  l2_loss = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)    
    
  #Loss function  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) + 0.000001 * l2_loss  
  # = tf.reduce_mean(
    #tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training
  train_prediction = tf.nn.softmax(logits2)

  #Predictions for the validation
  logits1= tf.matmul(tf_valid_dataset, weights_1) + biases_1
  hidden1 = tf.nn.relu(logits1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(hidden1, weights_2) + biases_2)
    
  #Predictions for the testing 
  logits1= tf.matmul(tf_test_dataset, weights_1) + biases_1
  hidden1 = tf.nn.relu(logits1)
  test_prediction = tf.nn.softmax(tf.matmul(hidden1, weights_2) + biases_2)

In [56]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_reshaped.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_reshaped[offset:(offset + batch_size), :]
    batch_labels = train_labels_reshaped[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels_reshaped))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_reshaped))

Initialized
Minibatch loss at step 0: 341.034119
Minibatch accuracy: 9.4%
Validation accuracy: 34.2%
Minibatch loss at step 500: 21.159004
Minibatch accuracy: 78.9%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 2.226162
Minibatch accuracy: 95.3%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 14.392435
Minibatch accuracy: 96.1%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 1.800854
Minibatch accuracy: 95.3%
Validation accuracy: 82.7%
Minibatch loss at step 2500: 0.426485
Minibatch accuracy: 98.4%
Validation accuracy: 82.4%
Minibatch loss at step 3000: 0.807494
Minibatch accuracy: 96.1%
Validation accuracy: 82.4%
Test accuracy: 91.2%


## Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [85]:
batch_size = 4096
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset_reshaped)
  tf_test_dataset = tf.constant(test_dataset_reshaped)
    # Variables in hidden layer.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases_1 = tf.Variable(tf.zeros([hidden1_units]))
  
  # Training computation.
  logits1= tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden1 = tf.nn.relu(logits1)
  
  #Variables in output layer
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  logits2 = tf.matmul(hidden1, weights_2) + biases_2

 #Regularization
  l2_loss = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)    
    
  #Loss function  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) + 0.000001 * l2_loss  
  # = tf.reduce_mean(
    #tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training
  train_prediction = tf.nn.softmax(logits2)

  #Predictions for the validation
  logits1= tf.matmul(tf_valid_dataset, weights_1) + biases_1
  hidden1 = tf.nn.relu(logits1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(hidden1, weights_2) + biases_2)
    
  #Predictions for the testing 
  logits1= tf.matmul(tf_test_dataset, weights_1) + biases_1
  hidden1 = tf.nn.relu(logits1)
  test_prediction = tf.nn.softmax(tf.matmul(hidden1, weights_2) + biases_2)

In [86]:
num_steps = 3001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset w  ithin the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_reshaped.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_reshaped[offset:(offset + batch_size), :]
    batch_labels = train_labels_reshaped[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels_reshaped))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_reshaped))

Initialized
Minibatch loss at step 0: 315.270691
Minibatch accuracy: 10.5%
Validation accuracy: 47.0%
Minibatch loss at step 500: 0.452890
Minibatch accuracy: 99.3%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 0.330083
Minibatch accuracy: 99.7%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 0.472208
Minibatch accuracy: 99.5%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 0.346026
Minibatch accuracy: 99.9%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 0.365123
Minibatch accuracy: 99.8%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.334400
Minibatch accuracy: 99.8%
Validation accuracy: 80.8%
Test accuracy: 87.5%


The training accuracy reached 99% quickly, but the validation accuracy was much below, around 81%. Overfitting happened in this case.

## Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.
What happens to our extreme overfitting case?

In [66]:
def createPlaceholder(shape=None):
    return tf.placeholder(tf.float32, shape)

def createWeights(size=None):
    return tf.Variable(tf.truncated_normal(size))

def createBiases(size=None):
    return tf.Variable(tf.zeros(size))  

In [82]:
batch_size = 4096
hidden1_units = 1024
graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  tf_train_dataset = createPlaceholder((batch_size, image_size * image_size))
  tf_train_labels = createPlaceholder((batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset_reshaped)
  tf_test_dataset = tf.constant(test_dataset_reshaped)
  #Dropout
  p_keep_input = tf.placeholder("float32")
  tf_train_dataset = tf.nn.dropout(tf_train_dataset, p_keep_input)
    
  # Variables in hidden layer.
  weights_1 = createWeights([image_size * image_size, hidden1_units])
  biases_1 = createBiases([hidden1_units])
  
  # Training computation.
  logits1= tf.matmul(tf_train_dataset, weights_1) + biases_1
  hidden1 = tf.nn.relu(logits1)
  p_hidden_input = tf.placeholder("float32")
  hidden1 = tf.nn.dropout(hidden1, p_hidden_input)
  
  #Variables in output layer
  weights_2 = createWeights([hidden1_units, num_labels])
  biases_2 = createBiases([num_labels])

  logits2 = tf.matmul(hidden1, weights_2) + biases_2

 #Regularization
  #l2_loss = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)    
    
  #Loss function  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training
  train_prediction = tf.nn.softmax(logits2)

  #Predictions for the validation
  logits1= tf.matmul(tf_valid_dataset, weights_1) + biases_1
  hidden1 = tf.nn.relu(logits1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(hidden1, weights_2) + biases_2)
    
  #Predictions for the testing 
  logits1= tf.matmul(tf_test_dataset, weights_1) + biases_1
  hidden1 = tf.nn.relu(logits1)
  test_prediction = tf.nn.softmax(tf.matmul(hidden1, weights_2) + biases_2)

In [84]:
num_steps = 3001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset w  ithin the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels_reshaped.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_reshaped[offset:(offset + batch_size), :]
    batch_labels = train_labels_reshaped[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, p_keep_input: 0.8, p_hidden_input: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels_reshaped))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_reshaped))

Initialized
Minibatch loss at step 0: 516.192627
Minibatch accuracy: 9.1%
Validation accuracy: 41.0%
Minibatch loss at step 500: 4.053629
Minibatch accuracy: 86.3%
Validation accuracy: 84.1%
Minibatch loss at step 1000: 2.274678
Minibatch accuracy: 88.0%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 1.616617
Minibatch accuracy: 89.8%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 1.281771
Minibatch accuracy: 90.5%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 0.814181
Minibatch accuracy: 90.9%
Validation accuracy: 84.7%
Minibatch loss at step 3000: 0.464619
Minibatch accuracy: 92.5%
Validation accuracy: 84.4%
Test accuracy: 92.1%


In this case, the training accuracy increased gradually, along with the validation accuracy. Overfitting was less serious here.

## Problem 4
Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.
One avenue you can explore is to add multiple layers.

In [87]:
def tf_deep_nn(regular=False, drop_out=False, lrd=False, hidden_layer_cnt=0):
    batch_size = 128
    hidden_node_count = 1024
    graph = tf.Graph()
    with graph.as_default():
        tf_train_dataset = createPlaceholder((batch_size, image_size * image_size))
        tf_train_labels = createPlaceholder((batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset_reshaped)
        tf_test_dataset = tf.constant(test_dataset_reshaped)
        
        # start weight
        hidden_stddev = np.sqrt(2.0 / 784)
        weights_start = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_node_count], stddev=hidden_stddev))
        biases_start = tf.Variable(tf.zeros([hidden_node_count]))
        # hidden layer weight
        weights = []
        biases = []
        hidden_cur_cnt = hidden_node_count
        for i in range(hidden_layer_cnt):
            if hidden_cur_cnt > 2:
                hidden_next_cnt = int(hidden_cur_cnt / 2)
            else:
                hidden_next_cnt = 2
            #Create weights and weights for each hidden layer
            hidden_stddev = np.sqrt(2.0 / hidden_cur_cnt)
            weights_hidden_layer = tf.Variable(tf.truncated_normal([hidden_cur_cnt, hidden_next_cnt], stddev=hidden_stddev))
            weights.append(weights_hidden_layer)
            bias_hidden_layer = tf.Variable(tf.zeros([hidden_next_cnt]))
            biases.append(bias_hidden_layer)
            hidden_cur_cnt = hidden_next_cnt
        # first wx + b
        y0 = tf.matmul(tf_train_dataset, weights_start) + biases_start
        # first relu
        hidden = tf.nn.relu(y0)
        hidden_drop = hidden
        # first DropOut
        keep_prob = 0.5
        if drop_out:
            hidden_drop = tf.nn.dropout(hidden, keep_prob)
        # first wx+b for valid
        valid_y0 = tf.matmul(tf_valid_dataset, weights_start) + biases_start
        valid_hidden = tf.nn.relu(valid_y0)
        # first wx+b for test
        test_y0 = tf.matmul(tf_test_dataset, weights_start) + biases_start
        test_hidden = tf.nn.relu(test_y0)

        # Calculate output for each hidden layer
        for i in range(hidden_layer_cnt):
            y1 = tf.matmul(hidden_drop, weights[i]) + biases[i]
            hidden_drop = tf.nn.relu(y1)
            if drop_out:
                keep_prob += 0.5 * i / (layer_cnt + 1)
                hidden_drop = tf.nn.dropout(hidden_drop, keep_prob)

            y0 = tf.matmul(hidden, weights[i]) + biases[i]
            hidden = tf.nn.relu(y0)

            valid_y0 = tf.matmul(valid_hidden, weights[i]) + biases[i]
            valid_hidden = tf.nn.relu(valid_y0)

            test_y0 = tf.matmul(test_hidden, weights[i]) + biases[i]
            test_hidden = tf.nn.relu(test_y0)

        # final weight
        weights_end = tf.Variable(tf.truncated_normal([hidden_cur_cnt, num_labels], stddev=hidden_stddev / 2))
        biases_end = tf.Variable(tf.zeros([num_labels]))
        # last wx + b
        logits = tf.matmul(hidden_drop, weights_end) + biases_end

        # predicts
        logits_predict = tf.matmul(hidden, weights_end) + biases_end
        valid_predict = tf.matmul(valid_hidden, weights_end) + biases_end
        test_predict = tf.matmul(test_hidden, weights_end) + biases_end

        l2_loss = 0
        # enable regularization
        if regular:
            l2_loss = tf.nn.l2_loss(weights_start) + tf.nn.l2_loss(weights_end)
            for i in range(len(weights)):
                l2_loss += tf.nn.l2_loss(weights[i])
                # l2_loss += tf.nn.l2_loss(biases[i])
            beta = 0.25 / batch_size
            beta = 1e-5
            l2_loss *= beta
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_loss

        # Optimizer.
        if lrd:
            cur_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
            starter_learning_rate = 0.4
            learning_rate = tf.train.exponential_decay(starter_learning_rate, cur_step, 100000, 0.96, staircase=True)
            optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=cur_step)
        else:
            optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits_predict)
        valid_prediction = tf.nn.softmax(valid_predict)
        test_prediction = tf.nn.softmax(test_predict)

    num_steps = 20001

    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
             # Pick an offset w  ithin the training data, which has been randomized.
            offset = (step * batch_size) % (train_labels_reshaped.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset_reshaped[offset:(offset + batch_size), :]
            batch_labels = train_labels_reshaped[offset:(offset + batch_size), :]
            feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if step % 500 == 0:
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(), valid_labels_reshaped))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_reshaped))

In [81]:
tf_deep_nn(regular=True, drop_out=True, lrd=False, hidden_layer_cnt=2)

Initialized
Minibatch loss at step 0: 2.344998
Minibatch accuracy: 8.6%
Validation accuracy: 18.3%
Minibatch loss at step 500: 0.714416
Minibatch accuracy: 81.2%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 0.524152
Minibatch accuracy: 84.4%
Validation accuracy: 85.3%
Minibatch loss at step 1500: 0.464589
Minibatch accuracy: 92.2%
Validation accuracy: 85.4%
Minibatch loss at step 2000: 0.420039
Minibatch accuracy: 92.2%
Validation accuracy: 87.2%
Minibatch loss at step 2500: 0.272450
Minibatch accuracy: 91.4%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 0.495406
Minibatch accuracy: 90.6%
Validation accuracy: 86.3%
Minibatch loss at step 3500: 0.359976
Minibatch accuracy: 92.2%
Validation accuracy: 86.7%
Minibatch loss at step 4000: 0.353752
Minibatch accuracy: 90.6%
Validation accuracy: 85.9%
Minibatch loss at step 4500: 0.184691
Minibatch accuracy: 94.5%
Validation accuracy: 87.2%
Minibatch loss at step 5000: 0.421083
Minibatch accuracy: 92.2%
Validation accuracy

Final accuracy improved to 94.1%. If we tuned the parameters, perhaps it would be higher.